In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import os
import pickle
import random

path = './'
weight_path = path+'weight/'

#### 모델 불러오기

In [2]:
# 모델 불러오기
model_dl = load_model(os.path.join(weight_path, 'model_dl.keras'))

# 전처리 도구 불러오기
with open(os.path.join(weight_path, 'vectorizer.pkl'), 'rb') as f:
    vectorizer = pickle.load(f)

with open(os.path.join(weight_path, 'gender_encoder.pkl'), 'rb') as f:
    gender_encoder = pickle.load(f)

with open(os.path.join(weight_path, 'relation_encoder.pkl'), 'rb') as f:
    relation_encoder = pickle.load(f)

with open(os.path.join(weight_path, 'age_scaler.pkl'), 'rb') as f:
    age_scaler = pickle.load(f)

with open(os.path.join(weight_path, 'encoder.pkl'), 'rb') as f:
    encoder = pickle.load(f)

with open(os.path.join(weight_path, 'direction_dict.pkl'), 'rb') as f:
    direction_dict = pickle.load(f)

with open(os.path.join(weight_path, 'gps_dict.pkl'), 'rb') as f:
    gps_dict = pickle.load(f)


In [7]:
def recommend_tourist_spots(keyword, gender, companion_relationship, age, direction, top_k=10, randomize=True):
    # 키워드 전처리 및 TF-IDF 벡터화
    def preprocess_text(text):
    # 특수문자 제거
        text = text.strip()
        text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]', '', text)
        return text

    keyword = preprocess_text(keyword)  # 텍스트 전처리 함수 (예: 특수 문자 제거)
    keyword_vector = vectorizer.transform([keyword]).toarray()  # TF-IDF 벡터화

    # 성별 인코딩
    gender_encoded = gender_encoder.transform([gender]).reshape(-1, 1)

    # 동반인 관계 One-Hot Encoding
    companion_relationship_df = pd.DataFrame([[companion_relationship]], columns=['동반인 관계'])
    companion_relationship_encoded = relation_encoder.transform(companion_relationship_df)

    # 나이 MinMax Scaling
    age_df = pd.DataFrame([[age]], columns=['나이'])
    age_scaled = age_scaler.transform(age_df)

    # 모델 입력 데이터를 결합
    input_data = [keyword_vector, gender_encoded, companion_relationship_encoded, age_scaled]

    # 예측 수행
    predictions = model_dl.predict(input_data, verbose=0)[0]


    # top_k 상위 항목 인덱스와 확률 순서 정렬
    sorted_indices = predictions.argsort()[::-1]  # 높은 확률 순서대로 정렬
    sorted_indices = [
        idx for idx in sorted_indices if direction_dict.get(encoder.inverse_transform([idx])[0]) == direction
    ]
    top_indices = sorted_indices[:top_k]  # 확률 상위 top_k 인덱스
    
   
    if randomize:
        # 20% 상위 0.5배 범위 중 랜덤 추출
        top_20_percent = list(map(int, random.sample(list(sorted_indices[:int(0.5 * top_k)]), int(0.2 * top_k))))

        # 40% 상위 0.5배~3배 범위 중 랜덤 추출
        top_40_percent_1 = list(map(int, random.sample(list(sorted_indices[int(0.5 * top_k):top_k * 3]), int(0.4 * top_k))))

        # 40% 상위 3배~10배 범위 중 랜덤 추출
        top_40_percent_2 = list(map(int, random.sample(list(sorted_indices[top_k * 3:top_k * 10]), int(0.2 * top_k))))

        # 나머지 20% 상위 10배 이후의 남은 범위 중 랜덤 추출
        remaining_indices = sorted_indices[top_k * 10:]
        extra_indices = list(map(int, random.sample(list(remaining_indices), min(int(0.2 * top_k), len(remaining_indices)))))

        # 추천 인덱스 합치기
        recommended_indices = top_20_percent + top_40_percent_1 + top_40_percent_2 + extra_indices
        recommended_indices = recommended_indices[:top_k]  # 최종 추천 인덱스를 top_k로 제한
    else:
        recommended_indices = top_indices


    # 인덱스를 관광지 이름으로 변환하고, GPS 좌표를 함께 포함
    recommendations = []
    for idx in recommended_indices:
        place_name = encoder.inverse_transform([idx])[0]
        gps_coordinates = gps_dict.get(place_name, {"주소" : None, "GPS 위도": None, "GPS 경도": None})
        address = gps_coordinates.get("주소")
        latitude = gps_coordinates.get("GPS 위도")
        longitude = gps_coordinates.get("GPS 경도")
        recommendations.append({
            "name": place_name,
            "address" : address,
            "latitude": latitude,
            "longitude": longitude
        })

    return recommendations

In [8]:
# 예시
keyword="자연, 소셜미디어, 스포츠"
gender="남"
companion_relationship="친구"
age=36
direction = 'northwest' 
recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction)
print(recommended_spots)

[{'name': '환상숲 곶자왈 공원', 'address': '제주특별자치도 제주시 한경면 녹차분재로 594-1', 'latitude': 33.3229067, 'longitude': 126.2630279}, {'name': '산지 등대', 'address': '제주특별자치도 제주시 사라봉동길 108-1', 'latitude': 33.5214246, 'longitude': 126.5456105}, {'name': '메종글래드 제주 더 파티오 풀', 'address': '제주특별자치도 제주시 노연로 80', 'latitude': 33.4852394, 'longitude': 126.4885322}, {'name': '사라봉공원', 'address': '제주특별자치도 제주시 사라봉동길 74', 'latitude': 33.5172842, 'longitude': 126.545546}, {'name': '내도바당 길', 'address': '제주특별자치도 제주시 일주서로 7469', 'latitude': 33.4964102, 'longitude': 126.4413505}, {'name': '도두항', 'address': '제주특별자치도 제주시 도두항길 22-2', 'latitude': 33.507398, 'longitude': 126.46655}, {'name': '제주칠성로', 'address': nan, 'latitude': 33.51324522, 'longitude': 126.5253755}, {'name': '노형수퍼마켙', 'address': '제주특별자치도 제주시 노형로 89', 'latitude': 33.4648249, 'longitude': 126.4553556}, {'name': '제주환상자전거길 1구간', 'address': nan, 'latitude': 33.49268656, 'longitude': 126.4275105}, {'name': '마이테르수영장', 'address': '제주특별자치도 제주시 애월읍 애원로 474-29', 'latitude':

In [9]:
# 예시
keyword="자연, 소셜미디어, 스포츠"
gender="남"
companion_relationship="친구"
age=36
top_k=10
direction = 'northwest'

for _ in range(5):
    recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction)
    print(recommended_spots)

[{'name': '협재해수욕장', 'address': '제주특별자치도 제주시 한림읍 한림로 329-10', 'latitude': 33.39385854, 'longitude': 126.2391575}, {'name': '용두암', 'address': '제주특별자치도 제주시 용두암길 15', 'latitude': 33.5161106, 'longitude': 126.5119571}, {'name': '메종글래드 제주 더 파티오 풀', 'address': '제주특별자치도 제주시 노연로 80', 'latitude': 33.4852394, 'longitude': 126.4885322}, {'name': '내도바당 길', 'address': '제주특별자치도 제주시 일주서로 7469', 'latitude': 33.4964102, 'longitude': 126.4413505}, {'name': '사라봉공원', 'address': '제주특별자치도 제주시 사라봉동길 74', 'latitude': 33.5172842, 'longitude': 126.545546}, {'name': '항파두리 항몽 유적지', 'address': '제주특별자치도 제주시 애월읍 항파두리로 50', 'latitude': 33.4527479, 'longitude': 126.4079226}, {'name': '진성배낚시', 'address': '제주특별자치도 제주시 한경면 노을해안로 1164-4', 'latitude': 33.3088234, 'longitude': 126.1649297}, {'name': '아르떼 뮤지엄 제주', 'address': '제주특별자치도 제주시 애월읍 어림비로 478', 'latitude': 33.3967005, 'longitude': 126.3450106}, {'name': '어승생승마장', 'address': '제주특별자치도 제주시 1100로 2659', 'latitude': 33.4256641, 'longitude': 126.490813}, {'name': '예나르 제주공예박

In [10]:
keyword="자연, 스포츠, 캠핑"
gender="남"
companion_relationship="친구"
age=36
direction = 'northeast' 
top_k = 15
recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)

recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)
for i in recommended_spots:
    print(i['관광지 명'])

함덕해수욕장
김녕해수욕장
만장굴
세화해변
비자림
절물자연휴양림
검멀레해변
제동목장
교래자연휴양림
한라생태숲
우도봉
우도산호해변
관곶
비양도 등대
제주 밭담 테마공원


In [11]:
keyword="자연 등반 둘레길"
gender="남"
companion_relationship="혼자"
age=25
direction = 'southeast' 

recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)
for i in recommended_spots:
    print(i['관광지 명'])

섭지코지
성산일출봉
자구리 문화 예술공원
쇠소깍
이중섭거리
마코다이브
오늘은 녹차 한 잔
서귀포 매일 올레시장
표선해수욕장
서복전시관
남원큰엉해변
신양섭지해수욕장
원앙폭포
제주 성읍마을
쇠소깍 전통 나룻배 카약 체험


In [8]:
keyword="힐링 소셜미디어"
gender="여"
companion_relationship="연인/배우자"
age=25
direction = 'northwest'

recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)
for i in recommended_spots:
    print(i['관광지 명'])

제주동문시장
제주동문재래시장
바이 제주
디앤디파트먼트 제주
제주시민속오일시장
제주동문공설시장
서문공설시장
도두몰
제주동문수산시장
귤귤스토어
가르송티미드 제주점
샵제주
소길별하
프라이탁 제주점
마켓 제주


In [9]:
keyword="유적지 역사유적박물관 자연 건강"
gender="여"
companion_relationship="혼자"
age=21
direction = 'northeast'

recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)
for i in recommended_spots:
    print(i['관광지 명'])

함덕해수욕장
해녀박물관
세화해변
비자림
김녕해수욕장
너븐숭이4.3기념관
절물자연휴양림
제주돌문화공원
올레길 18코스(제주 원도심-조천 올레)
만장굴
아침미소 목장
검멀레해변
스누피가든
우도봉
제주조천스위스마을


In [10]:
keyword="문화시설 친목 쇼핑 "
gender="여"
companion_relationship="친구"
age=21
direction = 'southwest'

recommended_spots = recommend_tourist_spots(keyword, gender, companion_relationship, age, direction,top_k, randomize=False)
for i in recommended_spots:
    print(i['관광지 명'])

제주아트서커스
기당미술관
오설록 티 뮤지엄
서귀포예술의전당
서귀포시 중앙도서관
제주실탄사격장
카멜리아힐
루나피크닉
산방산탄산온천
레이저아레나 엑스 제주점
루나폴
제주제트 중문점
안덕산방도서관
테디베어뮤지엄 제주점
무민랜드제주
